# Router
- Routing classifies and input and directs it to a specialized followup task.

![image.png](attachment:image.png)

In [1]:
pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 19.9 MB/s eta 0:00:00
  

In [19]:
from typing_extensions import Literal
from typing import TypedDict
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langgraph.graph import START, END, StateGraph
load_dotenv()

True

In [20]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [21]:
# Schema for structured output to use as routing logic
class Route(BaseModel):
    step: Literal["poem", "story", "song"] = Field(
        None, description = "The next step in the routing process."
    )

In [22]:
llm = ChatGroq(model = 'qwen-2.5-32b')
router = llm.with_structured_output(Route)

In [23]:
class State(TypedDict):
  input: str
  decision: str
  output: str

In [24]:
# Nodes

def llm_story(state: State):
  """Write a story"""
  print("Story function call")
  result = llm.invoke(state["input"])
  return {"output": result.content}

def llm_poem(state: State):
  """Write a poem"""
  print("Poem function call")
  result = llm.invoke(state["input"])
  return {"output": result.content}

def llm_song(state: State):
  """Write a 5 line song"""
  print("Song function call")
  result = llm.invoke(state["input"])
  return {"output": result.content}

# Router Node

def llm_router(state: State):
  """Route the input to appropriate function"""
  print("Router function called")

  decision = router.invoke(
      [
          SystemMessage(
              content="Route the input to story, song or poem on the user's request"
          ),
          HumanMessage(content=state["input"],)
      ]
  )
  return {"decision": decision.step}

In [25]:
def router_decision(state: State):
  if state["decision"] == "story":
    return "llm_story"
  elif state["decision"] == "poem":
    return "llm_poem"
  elif state["decision"] == "song":
    return "llm_song"

In [26]:
router_builder = StateGraph(State)

# Add Nodes

router_builder.add_node("llm_story", llm_story)
router_builder.add_node("llm_poem", llm_poem)
router_builder.add_node("llm_song",llm_song)
router_builder.add_node("llm_router", llm_router)

# Add edges and connect nodes

router_builder.add_edge(START, "llm_router")
router_builder.add_conditional_edges(
    "llm_router",
    router_decision,
    {
        "llm_story": "llm_story",
        "llm_poem": "llm_poem",
        "llm_song": "llm_song"
    },
)

router_builder.add_edge("llm_story", END)
router_builder.add_edge("llm_poem", END)
router_builder.add_edge("llm_song", END)

# Compile the workflow
router_workflow = router_builder.compile()

In [27]:
router_workflow

SSLError: HTTPSConnectionPool(host='mermaid.ink', port=443): Max retries exceeded with url: /img/JSV7aW5pdDogeydmbG93Y2hhcnQnOiB7J2N1cnZlJzogJ2xpbmVhcid9fX0lJQpncmFwaCBURDsKCV9fc3RhcnRfXyhbPHA+X19zdGFydF9fPC9wPl0pOjo6Zmlyc3QKCWxsbV9zdG9yeShsbG1fc3RvcnkpCglsbG1fcG9lbShsbG1fcG9lbSkKCWxsbV9zb25nKGxsbV9zb25nKQoJbGxtX3JvdXRlcihsbG1fcm91dGVyKQoJX19lbmRfXyhbPHA+X19lbmRfXzwvcD5dKTo6Omxhc3QKCV9fc3RhcnRfXyAtLT4gbGxtX3JvdXRlcjsKCWxsbV9wb2VtIC0tPiBfX2VuZF9fOwoJbGxtX3NvbmcgLS0+IF9fZW5kX187CglsbG1fc3RvcnkgLS0+IF9fZW5kX187CglsbG1fcm91dGVyIC0uLT4gbGxtX3N0b3J5OwoJbGxtX3JvdXRlciAtLi0+IGxsbV9wb2VtOwoJbGxtX3JvdXRlciAtLi0+IGxsbV9zb25nOwoJY2xhc3NEZWYgZGVmYXVsdCBmaWxsOiNmMmYwZmYsbGluZS1oZWlnaHQ6MS4yCgljbGFzc0RlZiBmaXJzdCBmaWxsLW9wYWNpdHk6MAoJY2xhc3NEZWYgbGFzdCBmaWxsOiNiZmI2ZmMK?type=png&bgColor=!white (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1020)')))

In [28]:
state = router_workflow.invoke({"input": "Write me a song on monitor just 5 lines"})
print(state["output"])

Router function called


APIConnectionError: Connection error.